In [2]:
 %tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import cv2,os,io
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0-rc1


In [0]:
import json

classes = ['REAL','FAKE']
class_list,name_of_files = [], []

with open("/content/drive/My Drive/DeepFakeFiles/train_data/metadata.json") as f:
    data = json.load(f)
    for video_id,info in data.items():
        if info['label'] == 'FAKE':
            class_list.append(1)
        else:
            class_list.append(0)
        name_of_files.append(video_id)

In [0]:
import glob
paths = glob.glob("/content/drive/My Drive/DeepFakeFiles/train_data/*.mp4")
print(len(paths))
labels = []
i=0
for f in paths:
  if os.path.basename(f) in name_of_files:
    labels.append(class_list[name_of_files.index(os.path.basename(f))])
print(labels,len(labels),"\n",name_of_files)  

In [0]:
from random import shuffle
c = list(zip(paths,labels))
shuffle(c)
paths,labels = zip(*c)
print(paths)
print(labels)

('/content/drive/My Drive/DeepFakeFiles/train_data/bzythlfnhq.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/benmsfzfaz.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/axwgcsyphv.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/dzwkmcwkwl.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/dboxtiehng.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/cbltdtxglo.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/awhmfnnjih.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/bwipwzzxxu.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/cwwandrkus.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/aevrfsexku.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/czfunozvwp.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/cdyakrxkia.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/ccmonzqfrz.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/eczrseixwq.mp4', '/content/drive/My Drive/DeepFakeFiles/train_data/bydaidkpdp.

In [0]:
train_paths = paths[0:int(0.8*len(paths))]
train_labels = labels[0:int(0.8*len(labels))]
val_paths = paths[int(0.8*len(paths)):int(1*len(paths))]
val_labels = labels[int(0.8*len(paths)):int(1*len(paths))]

In [0]:
def image_to_array(video,shape=(500,500),rescale=1/256.):
    nbframe = 5
    images = []
    frames = []

    cap = cv2.VideoCapture(video)
    while True:
        grabbed, frame = cap.read()
        if not grabbed:
          # print("End of video")
          break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, shape)

        frame = tf.keras.preprocessing.image.img_to_array(
            frame) * rescale
        frames.append(frame)
   

    jump = len(frames)//(nbframe+2)
    try:
      frames = frames[jump::jump][:nbframe]

    except Exception as exception:
      print(video)
      raise exception

    images.append(frames)

    return np.array(images).reshape(5,500,500,1)

In [0]:
import librosa
def audio_to_array(video):
  wave,sr = librosa.load(video)
  wave = wave[:][45000:150000]
  return wave.reshape(wave.shape[0],1)

In [0]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [0]:
import sys
train_filename = '/content/drive/My Drive/DeepFakeFiles/train.tfrecords'  # address to save the TFRecords file

# open the TFRecords file
writer = tf.io.TFRecordWriter(train_filename)

for i in range(len(train_paths)):
    # print how many images are saved every 1000 images
    if i % 10 == 0:
        print ('Train data: {}/{}'.format(i, len(train_paths)))
        sys.stdout.flush()

    # Load the image
    img = image_to_array(train_paths[i])
    audio = audio_to_array(train_paths[i])

    label = train_labels[i]

    # Create a feature
    feature = {'train/label': _int64_feature(label),
               'train/image': _bytes_feature(tf.compat.as_bytes(img.tostring())),
               'train/audio': _bytes_feature(tf.compat.as_bytes(audio.tostring()))}

    # Create an example protocol buffer
    example = tf.train.Example(features=tf.train.Features(feature=feature))

    # Serialize to string and write on the file
    writer.write(example.SerializeToString())

writer.close()
sys.stdout.flush()

Train data: 0/320
Train data: 10/320
Train data: 20/320
Train data: 30/320
Train data: 40/320
Train data: 50/320
Train data: 60/320
Train data: 70/320
Train data: 80/320
Train data: 90/320
Train data: 100/320
Train data: 110/320
Train data: 120/320
Train data: 130/320
Train data: 140/320
Train data: 150/320
Train data: 160/320
Train data: 170/320
Train data: 180/320
Train data: 190/320
Train data: 200/320
Train data: 210/320
Train data: 220/320
Train data: 230/320
Train data: 240/320
Train data: 250/320
Train data: 260/320
Train data: 270/320
Train data: 280/320
Train data: 290/320
Train data: 300/320
Train data: 310/320


In [0]:
val_filename = '/content/drive/My Drive/DeepFakeFiles/val.tfrecords'  # address to save the TFRecords file

# open the TFRecords file
writer = tf.io.TFRecordWriter(val_filename)

for i in range(len(val_paths)):
    # print how many images are saved every 1000 images
    if not i % 100:
        print ('Train data: {}/{}'.format(i, len(val_paths)))
        sys.stdout.flush()

    # Load the image
    # print(train_paths[i])
    img = image_to_array(val_paths[i])
    audio = audio_to_array(val_paths[i])

    label = val_labels[i]

    # Create a feature
    feature = {'val/label': _int64_feature(label),
               'val/image': _bytes_feature(tf.compat.as_bytes(img.tostring())),
               'val/audio': _bytes_feature(tf.compat.as_bytes(audio.tostring()))}

    # Create an example protocol buffer
    example = tf.train.Example(features=tf.train.Features(feature=feature))

    # Serialize to string and write on the file
    writer.write(example.SerializeToString())

writer.close()
print("Done")
sys.stdout.flush()

Train data: 0/80
Done


FOR TEST SET RUN THIS

In [0]:
import glob
test_paths = glob.glob("/content/drive/My Drive/DeepFakeFiles/test_videos/*.mp4")

In [0]:
file_path_test = "/content/drive/My Drive/DeepFakeFiles/test.tfrecords"

In [0]:
test_labels = [0 for _ in range(len(test_paths))]

In [0]:
import sys

# open the TFRecords file
writer = tf.io.TFRecordWriter(file_path_test)

for i in range(len(test_paths)):
    # print how many images are saved every 1000 images
    if i % 10 == 0:
        print ('Test data: {}/{}'.format(i, len(test_paths)))
        sys.stdout.flush()

    # Load the image
    img = image_to_array(test_paths[i])
    audio = audio_to_array(test_paths[i])

    label = test_labels[i]

    # Create a feature
    feature = {'test/label': _int64_feature(label),
               'test/image': _bytes_feature(tf.compat.as_bytes(img.tostring())),
               'test/audio': _bytes_feature(tf.compat.as_bytes(audio.tostring()))}

    # Create an example protocol buffer
    example = tf.train.Example(features=tf.train.Features(feature=feature))

    # Serialize to string and write on the file
    writer.write(example.SerializeToString())

writer.close()
sys.stdout.flush()

Test data: 0/400
Test data: 10/400
Test data: 20/400
Test data: 30/400
Test data: 40/400
Test data: 50/400
Test data: 60/400
Test data: 70/400
Test data: 80/400
Test data: 90/400
Test data: 100/400
Test data: 110/400
Test data: 120/400
Test data: 130/400
Test data: 140/400
Test data: 150/400
Test data: 160/400
Test data: 170/400
Test data: 180/400
Test data: 190/400
Test data: 200/400
Test data: 210/400
Test data: 220/400
Test data: 230/400
Test data: 240/400
Test data: 250/400
Test data: 260/400
Test data: 270/400
Test data: 280/400
Test data: 290/400
Test data: 300/400
Test data: 310/400
Test data: 320/400
Test data: 330/400
Test data: 340/400
Test data: 350/400
Test data: 360/400
Test data: 370/400
Test data: 380/400
Test data: 390/400


In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive
